In [ ]:
# Add project root to path and import required modules
import sys; sys.path.append('../..')
import inflation, mesh, sheet_meshing
from tri_mesh_viewer import OffscreenTriMeshViewer, TriMeshViewer as Viewer
import numpy as np
import utils, py_newton_optimizer, benchmark
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
import visualization

In [ ]:
# Load mesh vertices/edges and fused points (internal non-inflating regions)
V, E = mesh.load_raw('../data/ForwardDesign/hinge.obj')
fusedPts = list(np.loadtxt('../data/ForwardDesign/fusehingev3.txt').reshape((-1, 2)))
holePts = []  # No holes in this example

In [ ]:
# Create the mesh and inflatable sheet object
m, iwv, iwbv = sheet_meshing.forward_design_mesh(
    V, E, fusedPts, holePts, np.prod(utils.bbox_dims(V)[0:2]) / 1e4
)
isheet = inflation.InflatableSheet(m, iwv)

# Set material properties (ensure these attributes exist in your bindings)
isheet.thickness = 0.00015         # 0.15 mm (TPU nylon 70D)
isheet.youngModulus = 2.5e8        # 250 MPa

In [ ]:
# Visualize the 2D mesh and highlight fused vertices
# iwv is typically a boolean or index array for fused vertices
visualization.plot_2d_mesh(m, pointList=np.where(iwv)[0])

In [ ]:
# Set up offscreen viewer for video recording (no window will pop up)
oview = OffscreenTriMeshViewer(isheet, width=768, height=640, wireframe=True)

In [ ]:
# Define target points in 3D and find nearest mesh vertices for tracking
target_points = np.array([
    [5.0,  0.0, 0.0],
    [5.0, 20.0, 0.0],
    [5.0, 40.0, 0.0]
])

# Get rest positions and find nearest actual vertices
V_rest = np.array([v.flatten() for v in isheet.restWallVertexPositions()])
tree = KDTree(V_rest)
tracked_indices = [tree.query(p)[1] for p in target_points]

# Store rest positions for displacement calculation
rest_positions = {idx: V_rest[idx] for idx in tracked_indices}

print("Tracked indices:", tracked_indices)
print("Tracked coordinates:", [V_rest[i] for i in tracked_indices])

In [ ]:
# Set up dictionary to store trajectory over time
trajectory = {idx: [] for idx in tracked_indices}
framerate = 5  # Save every 5 iterations

def cb(it):
    if it % framerate == 0:
        for idx in tracked_indices:
            pos = isheet.getDeformedVtxPosition(idx, 0).flatten()
            trajectory[idx].append(pos)
        oview.update()  # Update offscreen viewer (for video)

In [ ]:
# Set up and run the simulation, recording video
benchmark.reset()
oview.recordStart('tracked_inflation3.mp4')  # Start recording

isheet.setUseTensionFieldEnergy(True)
isheet.setUseHessianProjectedEnergy(False)

opts = py_newton_optimizer.NewtonOptimizerOptions()
opts.niter = 500
isheet.pressure = 1

cr = inflation.inflation_newton(isheet, isheet.rigidMotionPinVars, opts, callback=cb)

benchmark.report()
oview.recordStop()  # End recording

In [ ]:
# Export the deformed mesh as an OBJ file for further use
def write_obj(path, V, F):
    with open(path, "w") as f:
        for v in V:
            f.write("v {:.6f} {:.6f} {:.6f}\n".format(*v))
        for face in F:
            f.write("f {} {} {}\n".format(*(face + 1)))  # OBJ is 1-indexed

V_raw, F_raw, *_ = isheet.visualizationGeometry()
V = np.array([v.flatten() for v in V_raw], dtype=np.float64)
F = np.array([f for f in F_raw], dtype=np.int32)
write_obj("inflated_mesh3.obj", V, F)

In [ ]:
# Plot Z displacement, X/Y displacement, and 3D trajectory for tracked vertices
fig = plt.figure(figsize=(18, 5))

# Z Displacement
ax1 = fig.add_subplot(1, 3, 1)
for idx in tracked_indices:
    traj = np.array(trajectory[idx])
    if traj.shape[0] > 1:
        z_disp = traj[:, 2] - rest_positions[idx][2]
        ax1.plot(range(len(traj)), z_disp, label=f"Vertex {idx} (Z)")
ax1.set_title("Z Displacement vs Iteration")
ax1.set_xlabel(f"Iteration (every {framerate} steps)")
ax1.set_ylabel("ΔZ from rest")
ax1.grid(True)
ax1.legend()

# X and Y Displacement
ax2 = fig.add_subplot(1, 3, 2)
for idx in tracked_indices:
    traj = np.array(trajectory[idx])
    if traj.shape[0] > 1:
        x_disp = traj[:, 0] - rest_positions[idx][0]
        y_disp = traj[:, 1] - rest_positions[idx][1]
        ax2.plot(range(len(traj)), x_disp, label=f"Vertex {idx} (X)")
        ax2.plot(range(len(traj)), y_disp, linestyle='--', label=f"Vertex {idx} (Y)")
ax2.set_title("X and Y Displacement vs Iteration")
ax2.set_xlabel(f"Iteration (every {framerate} steps)")
ax2.set_ylabel("ΔX / ΔY from rest")
ax2.grid(True)
ax2.legend()

# 3D Trajectory
ax3 = fig.add_subplot(1, 3, 3, projection='3d')
for idx in tracked_indices:
    traj = np.array(trajectory[idx])
    if traj.shape[0] > 1:
        ax3.plot(traj[:, 0], traj[:, 1], traj[:, 2], label=f"Vertex {idx}")
ax3.set_title("3D Trajectory of Tracked Vertices")
ax3.set_xlabel("X")
ax3.set_ylabel("Y")
ax3.set_zlabel("Z")
ax3.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plot absolute X, Y, Z positions vs iteration for each tracked vertex
fig, axs = plt.subplots(1, 3, figsize=(15, 4))

for idx in tracked_indices:
    traj = np.array(trajectory[idx])
    if traj.shape[0] <= 1:
        continue
    iterations = range(len(traj))
    axs[0].plot(iterations, traj[:, 0], label=f"Vertex {idx}")
    axs[1].plot(iterations, traj[:, 1], label=f"Vertex {idx}")
    axs[2].plot(iterations, traj[:, 2], label=f"Vertex {idx}")

# X
axs[0].set_title("X Position vs Iteration")
axs[0].set_xlabel(f"Iteration (every {framerate} steps)")
axs[0].set_ylabel("X Position")
axs[0].grid(True)
axs[0].legend()

# Y
axs[1].set_title("Y Position vs Iteration")
axs[1].set_xlabel(f"Iteration (every {framerate} steps)")
axs[1].set_ylabel("Y Position")
axs[1].grid(True)
axs[1].legend()

# Z
axs[2].set_title("Z Position vs Iteration")
axs[2].set_xlabel(f"Iteration (every {framerate} steps)")
axs[2].set_ylabel("Z Position")
axs[2].grid(True)
axs[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
# On-screen 3D viewer for interactive inspection (run in a separate cell)
viewer = Viewer(isheet, wireframe=True)
viewer.show()